In [ ]:
import os
import shutil
import mlflow
from glob import glob
from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv("../.env")

## ローカルのモデルパラメータのrun_idを取得

In [ ]:
localPaths = glob('../checkpoint/*')
localRunIds = set([path.rsplit('/', 1)[1] for path in localPaths])

print( len(localRunIds) )

## 存在するrun_idを取得

In [ ]:
activeExperimentIds = [experiment.experiment_id for experiment in mlflow.search_experiments(view_type="ACTIVE_ONLY")]
print(activeExperimentIds)

activeRuns = mlflow.search_runs(experiment_ids=activeExperimentIds, run_view_type=1)
activeRunIds = set(activeRuns["run_id"])
print( len(activeRunIds) )

## MLflow上で削除されたrunのモデルパラメータを削除する

In [ ]:
removedRunIds = localRunIds - activeRunIds
print( len(removedRunIds) )

In [ ]:
for runId in tqdm(removedRunIds):
    
    path = f'../checkpoint/{runId}'
    shutil.rmtree(path)